In [1]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

57344/57026 [==============================] - 1s 10us/step


In [2]:
train_data.shape

(404, 13)

In [3]:
test_data.shape

(102, 13)

In [4]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [8]:
train_data

array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
        3.95380e+02, 3.11000e+00],
       [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.75520e+02, 3.26000e+00],
       ...,
       [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
        3.62250e+02, 7.83000e+00],
       [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        2.61950e+02, 1.57900e+01],
       [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
        3.76700e+02, 4.38000e+00]])

In [9]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

In [14]:
from keras import models
from keras import layers
def build_model():
     model = models.Sequential() 
     model.add(layers.Dense(64, activation='relu',
     input_shape=(train_data.shape[1],)))
     model.add(layers.Dense(64, activation='relu'))
     model.add(layers.Dense(1))
     model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
     return model
'''
由于数据点很少，验证集会非常小（比如大约
100 个样本）。因此，验证分数可能会有很大波动，这取决于你所选择的验证集和训练集。也就
是说，验证集的划分方式可能会造成验证分数上有很大的方差，这样就无法对模型进行可靠的
评估。
在这种情况下，最佳做法是使用 K 折交叉验证（见图 3-11）。这种方法将可用数据划分为 K
个分区（K 通常取 4 或 5），实例化 K 个相同的模型，将每个模型在 K-1 个分区上训练，并在剩
下的一个分区上进行评估。模型的验证分数等于 K 个验证分数的平均值。这种方法的代码实现
很简单
'''
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
     print('processing fold #', i)
     val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples] 
     val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
        
     partial_train_data = np.concatenate( 
                                         [train_data[:i * num_val_samples],
                                         train_data[(i + 1) * num_val_samples:]], 
                                         axis=0)
     partial_train_targets = np.concatenate(
                                         [train_targets[:i * num_val_samples],
                                         train_targets[(i + 1) * num_val_samples:]], 
                                         axis=0)
     model = build_model() 
     model.fit(partial_train_data, partial_train_targets, 
                 epochs=num_epochs, batch_size=1, verbose=0)
     val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0) 
     all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [15]:
 np.mean(all_scores)

2.37330424785614

In [ ]:
'''
 回归问题使用的损失函数与分类问题不同。回归常用的损失函数是均方误差（MSE）。
 同样，回归问题使用的评估指标也与分类问题不同。显而易见，精度的概念不适用于回
归问题。常见的回归指标是平均绝对误差（MAE）。
 如果输入数据的特征具有不同的取值范围，应该先进行预处理，对每个特征单独进行
缩放。
 如果可用的数据很少，使用 K 折验证可以可靠地评估模型。
 如果可用的训练数据很少，最好使用隐藏层较少（通常只有一到两个）的小型网络，以
避免严重的过拟合。
'''